# Introduction

This notebook provides some tools on how to profile and time python code. This can 
help to make your code a lot faster. However, there are certain limits on how fast 
you can rewrite your code. For additional speedup you must use HPC libraries. I will 
cover two in this notebook: Numba and Jax. Both have their advantages, not one 
dominates the other and it is up to hardware, personal use and familiarity which is 
more efficient to use.

## Part 1: Profiling and timing

In [1]:
import numpy as np
import time

In [2]:
def utility(x, floor):
    floored_consumption = np.where(x < floor, floor, x)
    return np.log(floored_consumption)

In [3]:
max_grid = 1_000_000
num_grid = 1_000_000
consumption = np.linspace(1, max_grid, num_grid)

#### Timing

In [4]:
# Naive timing
tic = time.time()
utility(consumption, 1)
toc = time.time()
print(f"Time elapsed: {toc - tic: .5f} seconds")

Time elapsed:  0.01913 seconds


In [5]:
# Timeit module
%timeit utility(consumption, 1)

10.2 ms ± 268 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Profiling

In [6]:
%load_ext snakeviz

In [7]:
%snakeviz -t utility(consumption, 1)

 
*** Profile stats marshalled to file '/tmp/tmpn2n9jckl'.
Opening SnakeViz in a new tab...


In [1]:
import numpy as np
import numba as nb

In [2]:
def utility_log(x):
    return np.log(x)

In [3]:
max_grid = 1000
num_grid = 1000000
consumption = np.linspace(1, max_grid, num_grid)

In [4]:
%timeit utility_log(consumption)

4.51 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
@nb.jit(nopython=True)
def utility_log_numba(x):
    for i in range(len(x)):
        x[i] = np.log(x[i])
    return x

In [6]:
utility_log_numba(consumption)
%timeit utility_log_numba(consumption)

3.86 ms ± 43.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Introduce an consumption floor

In [7]:
def utility_log_floor(x, floor):
    return np.log(np.maximum(x, floor))

@nb.njit()
def utility_log_floor_numba(x, floor):
    for i in range(len(x)):
        x[i] = np.log(max(x[i], floor))
    return x

In [8]:
%timeit utility_log_floor(consumption, 1)

7.26 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
utility_log_floor(consumption, 1)
%timeit utility_log_floor_numba(consumption, 1)

3.91 ms ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
